In [1]:
import os, sys
import numpy as np
import pandas as pd
import lightgbm
import json

## Prepare Data

In [2]:
# def infer_categorical_features(dataset):
#     categorical_features = set([])
#     for column in dataset.columns:
#         if dataset[column].dtype == 'object':
#             categorical_features.add(column)
#     return categorical_features
            
def label_encode(dataset, categorical_features):
    dataset_le = dataset.copy()
    for column in dataset_le.columns:
        if column in categorical_features:
            dataset_le[column] = dataset_le[column].astype('category')
            dataset_le[column] = dataset_le[column].cat.codes.astype(np.int32)
    return dataset_le

def load_atk_train_valid_test( atk_train_file, atk_valid_file, atk_test_file, 
                               train_split=0.6, valid_split=0.2,
                               force=False):
    
    
    if  ( force or 
          not os.path.exists(atk_train_file+".cat.bz2") or
          not os.path.exists(atk_valid_file+".cat.bz2") or
          not os.path.exists(atk_test_file+".cat.bz2") or 
          not os.path.exists(atk_train_file+".cat.json") ):
    
        print ("Pre-processing original files...")

        print ("Loading:", atk_train_file)
        print ("Loading:", atk_valid_file)
        print ("Loading:", atk_test_file)

        train = pd.read_csv(atk_train_file)
        valid = pd.read_csv(atk_valid_file)
        test  = pd.read_csv(atk_test_file)
        
        print ("Train/Valid/Test sizes:", train.shape, valid.shape, test.shape)
        print ("Train/Valid/Test split: {:.2f} {:.2f} {:.2f}"
                   .format( train.shape[0]/(train.shape[0]+valid.shape[0]+test.shape[0]),
                            valid.shape[0]/(train.shape[0]+valid.shape[0]+test.shape[0]),
                            test.shape[0] /(train.shape[0]+valid.shape[0]+test.shape[0]) ) )

        # concat to process correctly label encoding
        full = pd.concat( [train, valid, test] )

        # get index of categorical features (-1 because of instance_id)
        cat_fx = full.columns.values[ np.where(full.dtypes=='object')[0] ]
        cat_fx = list(cat_fx)    
        full = label_encode(full, cat_fx)
        with open(atk_train_file+".cat.json", 'w') as fp:
            json.dump(cat_fx, fp)
        print ("CatFX:", cat_fx)

        # split-back into train valid test
        train_size = int( full.shape[0]*train_split )
        valid_size = int( full.shape[0]*valid_split )
        train_cat = full.iloc[0:train_size,:]
        valid_cat = full.iloc[train_size:train_size+valid_size,:]
        test_cat  = full.iloc[train_size+valid_size:,:]    

        print ("Train/Valid/Test sizes:", train_cat.shape, valid_cat.shape, test_cat.shape)
        print ("Train/Valid/Test split: {:.2f} {:.2f} {:.2f}"
                   .format( train_cat.shape[0]/(train_cat.shape[0]+valid_cat.shape[0]+test_cat.shape[0]),
                            valid_cat.shape[0]/(train_cat.shape[0]+valid_cat.shape[0]+test_cat.shape[0]),
                            test_cat.shape[0] /(train_cat.shape[0]+valid_cat.shape[0]+test_cat.shape[0]) ) )

        # save to file
        print ("Saving processed files *.cat.bz2")
        train_cat.to_csv(atk_train_file+".cat.bz2", compression="bz2", index=False)
        valid_cat.to_csv(atk_valid_file+".cat.bz2", compression="bz2", index=False)
        test_cat.to_csv (atk_test_file+".cat.bz2",  compression="bz2", index=False)
        
    else:
        print ("Loading pre-processed files...")

        train_cat = pd.read_csv(atk_train_file+".cat.bz2")
        valid_cat = pd.read_csv(atk_valid_file+".cat.bz2")
        test_cat  = pd.read_csv(atk_test_file+".cat.bz2")
        
        with open(atk_train_file+".cat.json", 'r') as fp:
            cat_fx = json.load(fp)
    
    # return data
    return train_cat, valid_cat, test_cat, cat_fx


## Adversarial Boosting

In [3]:
def AdvBoosting_gen_data(model, data, groups):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    groups : grouping of same attacked instance 
    returns the new data matrix and new groups
    
    WARNING: currently works only for binary classification
    '''
    # score the datataset
    labels = data[:,-1]
    
    predictions = model.predict(data[:,:-1]) # exclude labels
    # binarize
    predictions = (predictions>0).astype(np.float)
    predictions = 2*predictions - 1
    
    # check mispredictions
    matchings = labels * predictions
    
    # select original data + attacked instances
    new_selected = [] # id of selected instances
    new_groups   = []
    
    offset = 0
    for g in groups:
        if g==0:
            print ("Error !!!!")
        elif g==1:
            # there are no attacks, just add original
            new_selected += [offset]
            new_groups   += [1]
        else:
            # get a slice of the matching scores
            g_matchings = matchings[offset:offset+g]

            # most misclassified (smallest margin)
            # skip original
            adv_instance = np.argmin(g_matchings[1:])+1

            # add original and adversarial
            new_selected += [offset, adv_instance]
            new_groups   += [2]
        
        offset += g
    
    new_dataset = data[new_selected,:]
    
    return new_dataset, new_groups

In [4]:
# Our custom metrics
def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False


def optimize_log_loss(preds, train_data):
    labels = train_data.get_label()
    exp_pl = np.exp(preds * labels)
    # http://www.wolframalpha.com/input/?i=differentiate+log(1+%2B+exp(-kx)+)
    grads = -labels / (1.0 +  exp_pl)  
    # http://www.wolframalpha.com/input/?i=d%5E2%2Fdx%5E2+log(1+%2B+exp(-kx)+)
    hess = labels**2 * exp_pl / (1.0 + exp_pl)**2 

    # this is to optimize average logloss
    norm = 1.0/len(preds)
    grads *= norm
    hess *= norm
    
    return grads, hess


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))

            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  

    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={}]'.format(alpha), weighted_loss, False

def optimize_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    grads = np.zeros_like(labels, dtype=np.float64)
    hess = np.zeros_like(grads)
    
    if attack_lens is not None:

        norm = 1.0 / float(len(attack_lens))

        offset = 0
        for atk in attack_lens:
            exp_pl = np.exp(- preds[offset:offset+atk] * labels[offset:offset+atk])

            inv_sum = 1.0 / np.sum(1.0 + exp_pl)

            x_grad = inv_sum * exp_pl

            grads[offset:offset+atk] = norm * x_grad * (- labels[offset:offset+atk])
            hess[offset:offset+atk]  = norm * x_grad * (1.0 - x_grad)

            offset += atk    
    
    return grads, hess

def optimize_non_interferent_log_loss(preds, train_data, alpha=1.0):
    # binary logloss under maximal attack
    grads_uma, hess_uma = optimize_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    grads_plain, hess_plain = optimize_log_loss(preds, train_data)
    
    # combine the above two losses together
    grads = alpha*grads_uma + (1.0-alpha)*grads_plain
    hess  = alpha*hess_uma  + (1.0-alpha)*hess_plain
    
    return grads, hess



def AdvBoosting_extend_model(data, cat_fx, input_model=None, num_trees=1, params=None):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    returns the new model (is model modified inplace?)
    '''
    
    if cat_fx is None or len(cat_fx)==0:
        cat_fx = "auto"

    lgbm_train = lightgbm.Dataset(data=data[:,:-1], 
                                  label=data[:,-1],
                                  categorical_feature = cat_fx)
    
    lgbm_info = {}
    lgbm_model = lightgbm.train(params, lgbm_train, 
                                num_boost_round = num_trees, 
                                init_model = input_model,
                                fobj = optimize_log_loss, 
                                feval = avg_log_loss,
                                evals_result = lgbm_info,
                                valid_sets   = [lgbm_train], 
                                valid_names  = ['adv-train'],
                                verbose_eval=10)

    return lgbm_model, lgbm_info

In [5]:
def AdvBoosting( atk_train, trees, 
                 cat_fx,
                 params,
                 output_model_file,
                 partial_save=10, 
                 adv_rounds=1):
    ''' 
    atk_data: full dataset including all valid attacks
    atk_groups: lenght of each attack set
    trees: total number of trees to be produced
    adv_rounds: adversarial instance injecting frequency
    '''
    # temp lgbm file
    temp = output_model_file+".tmp"
    
    # get groups and remove instance ids
    atk_groups = atk_train['instance_id'].value_counts().sort_index().values
    atk_train.drop('instance_id', axis=1, inplace=True)
    
    # get index of categorical features 
    cat_fx = np.where(atk_train.columns.isin(cat_fx) )[0]
    cat_fx = list([int(x) for x in cat_fx])  
    # print ("CatFX:", atk_train.columns.values[cat_fx])

    # prepare data (avoiding pandas)
    atk_data   = atk_train.values

    # train first trees
    original_ids = np.cumsum(atk_groups[:-1])
    original_ids = np.insert(original_ids, 0, 0)
    
    model, model_info = AdvBoosting_extend_model( atk_data[original_ids, :], 
                                                  cat_fx=cat_fx,
                                                  input_model=None, 
                                                  num_trees=adv_rounds, 
                                                  params=params )
    
    
    # train remaining trees
    for t in range(adv_rounds+1, trees+1, adv_rounds):
        # attack dataset
        adv_data, adv_offsets = AdvBoosting_gen_data( model, atk_data, atk_groups )
        
        # train additional trees
        model.save_model(temp)
        model, model_info = AdvBoosting_extend_model( adv_data, 
                                                      cat_fx=cat_fx,
                                                      input_model=temp, 
                                                      num_trees=adv_rounds, 
                                                      params=params)
        # save partial model
        if t%partial_save==0 and t!=trees:
            partial_filename = "{}.T{:03d}.lgbm".format(output_model_file, t)
            model.save_model( filename=partial_filename )
            
    model.save_model(filename=output_model_file)
    
    return model, model_info

# Scripting wrapper

In [6]:
def run_training( atk_train_file, atk_valid_file, atk_test_file,
                  output_model_file,
                  num_trees,
                  learning_rate,
                  num_leaves,
                  partial_save,
                  adversarial_rounds):

    # load train/valid/test
    train, valid, test, cat_fx = load_atk_train_valid_test(atk_train_file, atk_valid_file, atk_test_file)
    
    # train params
    lgbm_params = { 'learning_rate': float(learning_rate), 
                    'num_leaves': int(num_leaves)} 

    # start training
    return AdvBoosting(  train, 
                         trees=num_trees, 
                         cat_fx = cat_fx,
                         output_model_file=output_model_file, 
                         adv_rounds=adversarial_rounds, 
                         partial_save=partial_save, 
                         params=lgbm_params)

# Gradient Boosting Baseline

In [10]:
def train_gradient_boosting_baseline( train_file, valid_file, test_file,
                                      output_model_file ):
    
    # load train/valid/test
    train, valid, test, cat_fx = load_atk_train_valid_test(train_file, valid_file, test_file)

    # get index of categorical features 
    cat_fx = np.where(train.columns.isin(cat_fx) )[0]
    cat_fx = list([int(x) for x in cat_fx])  
    print ("CatFX:", train.columns.values[cat_fx])
    
    # datasets
    lgbm_train = lightgbm.Dataset(data=train.values[:,:-1], 
                                  label=train.values[:,-1],
                                  categorical_feature = cat_fx)
    
    lgbm_valid = lightgbm.Dataset(data=valid.values[:,:-1], 
                                  label=valid.values[:,-1],
                                  categorical_feature = cat_fx)

    # train params
    lgbm_params = { 'learning_rate': 0.1, 
                    'num_leaves': 16} 
    lgbm_info = {}
    lgbm_model = lightgbm.train(lgbm_params, lgbm_train, 
                                num_boost_round = 50,
                                fobj            = optimize_log_loss, 
                                feval           = avg_log_loss,
                                evals_result    = lgbm_info,
                                valid_sets      = [lgbm_train, lgbm_valid], 
                                valid_names     = ['train', 'valid'],
                                verbose_eval    = 10)

    print (lgbm_info)
    print (len ( lgbm_info['valid']['avg_binary_log_loss']) )
    
    
    
    
train_gradient_boosting_baseline ( "../data/census/train_ori.csv.bz2",
                                   "../data/census/valid_ori.csv.bz2",
                                   "../data/census/test_ori.csv.bz2",
                                   "../out/models/GBDT_census.lgbm" )  

Loading pre-processed files...
CatFX: ['workclass' 'marital_status' 'occupation' 'relationship' 'race' 'sex'
 'native_country']


/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	train's avg_binary_log_loss: 0.442694	valid's avg_binary_log_loss: 0.443485
[20]	train's avg_binary_log_loss: 0.383888	valid's avg_binary_log_loss: 0.384591
[30]	train's avg_binary_log_loss: 0.364054	valid's avg_binary_log_loss: 0.365226
[40]	train's avg_binary_log_loss: 0.354723	valid's avg_binary_log_loss: 0.35629
[50]	train's avg_binary_log_loss: 0.332285	valid's avg_binary_log_loss: 0.336143
{'train': defaultdict(<class 'list'>, {'avg_binary_log_loss': [0.6442831012057314, 0.6043478691937245, 0.5712039382536149, 0.5429154323835782, 0.5194995815960616, 0.49926751235772854, 0.4818859720707152, 0.4670586329204811, 0.45398464711676156, 0.44269372914143423, 0.4326994349622958, 0.4242787202485628, 0.4165668471107094, 0.40984903164484426, 0.40423865203413867, 0.3990045596889301, 0.39457656403367825, 0.39066311150609556, 0.3869512093246306, 0.38388825406014965, 0.38107445998894607, 0.37843677935834447, 0.3762391990774234, 0.3738217495960891, 0.37205918526098186, 0.3704253868418409, 0.

# Adversatial Boosing Training

In [84]:
run_training( "../data/census/train_B5.csv.bz2", 
              "../data/census/valid_B5.csv.bz2",
              "../data/census/test_B5.csv.bz2",
              "../out/models/adv_boosting.lgbm",
              num_trees=10,
              num_leaves=16,
              learning_rate=0.1,
              partial_save=10,
              adversarial_rounds=1)



Loading pre-processed files...


/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	adv-train's avg_binary_log_loss: 0.3392


In [70]:
!ls -lht ../data/census/

total 20M
-rw-rw-r-- 1 lucchese lucchese 305K Feb 18 18:46 test_B5.csv.bz2.cat.bz2
-rw-rw-r-- 1 lucchese lucchese 307K Feb 18 18:46 valid_B5.csv.bz2.cat.bz2
-rw-rw-r-- 1 lucchese lucchese 912K Feb 18 18:46 train_B5.csv.bz2.cat.bz2
-rw-rw-r-- 1 lucchese lucchese   94 Feb 18 18:46 train_B5.csv.bz2.cat.json
-rw-rw-r-- 1 lucchese lucchese 503K Feb 18 18:41 test_B5.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 100K Feb 18 18:41 valid_B5.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 902K Feb 18 18:41 train_B5.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 103K Feb 18 18:13 test_ori.csv.bz2
-rw-rw-r-- 1 lucchese lucchese  23K Feb 18 18:13 valid_ori.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 183K Feb 18 18:13 train_ori.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 902K Feb 18 14:11 test_B15.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 179K Feb 18 14:11 valid_B15.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 1.6M Feb 18 14:11 train_B15.csv.bz2
-rw-rw-r-- 1 lucchese lucchese 1.1M Feb 18 09:52 test_strong_att.csv.bz2.cat.bz2
-rw-rw-r-- 1 luc

In [102]:
# run_training( "../data/census/train_B5.csv.bz2", 
#               "../data/census/valid_B5.csv.bz2",
#               "../data/census/test_B5.csv.bz2",
#               "../out/models/debug.lgbm",
#               num_trees=10,
#               learning_rate=0.1,
#               partial_save=1000,
#               adversarial_rounds=10)

Loading pre-processed files...
CatFX: ['workclass' 'marital_status' 'occupation' 'relationship' 'race' 'sex'
 'native_country']
[10]	adv-train's avg_binary_log_loss: 0.442759


In [10]:
#!ls -lht ../out/models

In [11]:
# debug running time
#%load_ext line_profiler
# %lprun -f AdvBoosting  AdvBoosting(train, trees=3)
# %lprun -f AdvBoosting_gen_data  AdvBoosting(train, trees=3)

In [12]:
#!rm ../out/models/*.lgbm